In [10]:
# 'dataset' holds the input data for this script

import pandas as pd
import numpy as np

dataset = pd.DataFrame({"A":[1, 1, 1, 2, 2, 3, 3, 3, 3]
                        , "B":[0, 0, 0, 5, 5, 6, 6, 6, 6]
                        , "D":[1, 2, 3, 4, 5, 6, 7, 8, 9]
                        , "treshold":[0, 1, 2, 0, 1, 0, 1, 2, 3]})

dataset

,A,B,D,treshold
0,1,0,1,0
1,1,0,2,1
2,1,0,3,2
3,2,5,4,0
4,2,5,5,1
5,3,6,6,0
6,3,6,7,1
7,3,6,8,2
8,3,6,9,3


In [24]:
dataset = dataset.sort_values(by=["A", "B", "treshold"])

dataset["AuROC"] = 0

for i in np.arange(dataset.shape[0]):
    if dataset.loc[i, "treshold"] == 0:
        dataset.loc[i, "AuROC"] = dataset.loc[i, "D"]
    else:
        dataset.loc[i, "AuROC"] = dataset.loc[i-1, "AuROC"] + dataset.loc[i, "D"]

dataset["AuROC"] = dataset.join(dataset.groupby(by=["A", "B"])["AuROC"].max(), on=["A", "B"], lsuffix='_acum')["AuROC"]
dataset

,A,B,D,treshold,AuROC
0,1,0,1,0,6
1,1,0,2,1,6
2,1,0,3,2,6
3,2,5,4,0,9
4,2,5,5,1,9
5,3,6,6,0,30
6,3,6,7,1,30
7,3,6,8,2,30
8,3,6,9,3,30


In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import scipy
import math
import librosa
import heartpy as hp
import datetime
import os
import re

In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                        CSVLogger, EarlyStopping)
from csv import writer
from datetime import datetime

In [18]:
tf.__version__

'2.13.0'

In [3]:
from load_dataset import create_dataset
from callback_save_files import TrainingCallback, FitCallback

In [4]:
WORKING_DIRECTORY = 'C:/CVUT/Diplomka/'
logs_dir = f'{WORKING_DIRECTORY}Data/logs/'
file_name = '20231203 Channel-desease dependency'
task_name = ''

# Import data

In [5]:
val_filepath = f'{WORKING_DIRECTORY}Data/CODE-15/val'
train_filepath = f'{WORKING_DIRECTORY}Data/CODE-15/train'

val_Y = create_dataset('val_Y', val_filepath,0)
val_F_X = create_dataset('val_I_F_X_m', val_filepath,0)

val_dataset = tf.data.Dataset.zip((val_F_X, val_Y))

train_Y = create_dataset('train_Y', train_filepath,0)
train_F_X = create_dataset('train_I_F_X_m', train_filepath,0)

train_dataset = tf.data.Dataset.zip((train_F_X, train_Y))

Undersampling for speed improvement

In [6]:
rate_undersampling = 0.1

In [7]:
@tf.autograph.experimental.do_not_convert
def random_undersampling(x, y):
    return tf.random.uniform((1, 1), 0, 1)[0, 0] < rate_undersampling

In [8]:
short_train_dataset = train_dataset.filter(lambda x, y: tf.reduce_sum(y)==0).filter(random_undersampling).concatenate(train_dataset.filter(lambda x, y: tf.reduce_sum(y)>0))

Cause: could not parse the source code of <function <lambda> at 0x0000021BA003CEE0>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
lambda x, y: tf.reduce_sum(y) > 0

Match 1:
lambda x, y: tf.reduce_sum(y) == 0

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x0000021BA003CEE0>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
lambda x, y: tf.reduce_sum(y) > 0

Match 1:
lambda x, y: tf.reduce_sum(y) == 0

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 

In [9]:
short_train_dataset = short_train_dataset.shuffle(short_train_dataset.cardinality(), reshuffle_each_iteration  = False)

In [10]:
del train_dataset

In [11]:
inst_count = 0
for inst in short_train_dataset:
    inst_count += 1

In [12]:
inst_count

2724

# CNN

In [13]:
lr = 0.001

In [17]:
        input_F_X = tf.keras.Input(shape=(500,12,), name='CNN_input_F_X')
        Flat = tf.keras.layers.Flatten(name='Flat')(input_F_X)
        D_1 = tf.keras.layers.Dense(120, activation = 'linear')(Flat)
        D_2 = tf.keras.layers.Dense(90 , activation = 'linear')(D_1)
        D_3 = tf.keras.layers.Dense(60 , activation = 'linear')(D_2)
        D_4 = tf.keras.layers.Dense(30 , activation = 'linear')(D_3)
        outputs = tf.keras.layers.Dense(6, activation='sigmoid')(D_4)
        model = tf.keras.models.Model(inputs=input_F_X
                                      , outputs=outputs)
        #model.summary()

        model.compile(loss=tf.keras.losses.BinaryCrossentropy() #'binary_crossentropy'
                 , optimizer=tf.keras.optimizers.Adam(lr)
                 , metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')
                            , tf.keras.metrics.Precision(name='precision')
                            , tf.keras.metrics.Recall(name='recall')
                            , tf.keras.metrics.F1Score(name='f1_score', average='micro')])

        # CALLBACKS
        callbacks = [ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.1,
                                           patience=6,
                                           min_lr=lr / 100),
                         EarlyStopping(patience=11,  # Patience should be larger than the one in ReduceLROnPlateau
                                       min_delta=0.00001),
                    FitCallback(logs_dir, file_name, val_dataset, 'All channels')
                    ]

        model.fit( short_train_dataset.batch(8).repeat(70),
                epochs=70,
                initial_epoch=0,
                steps_per_epoch = np.floor(inst_count/8),
                #batch_size = 8,
                callbacks = callbacks,
                #callbacks=[myCallback(init_vars)], 
                validation_data=(val_dataset.batch(8)),
                verbose=1)

<_ZipDataset element_spec=(TensorSpec(shape=(500, 12), dtype=tf.float64, name=None), TensorSpec(shape=(6,), dtype=tf.float64, name=None))>
Epoch 1/70
340/340 [==============================] - 4s 7ms/step - loss: 4.8770 - binary_accuracy: 0.8704 - precision: 0.3671 - recall: 0.3891 - f1_score: 0.3717 - val_loss: 0.3356 - val_binary_accuracy: 0.9620 - val_precision: 0.2060 - val_recall: 0.2929 - val_f1_score: 0.0973 - lr: 0.0010
Epoch 2/70
340/340 [==============================] - 4s 12ms/step - loss: 0.6895 - binary_accuracy: 0.8941 - precision: 0.4879 - recall: 0.4673 - f1_score: 0.4366 - val_loss: 0.2694 - val_binary_accuracy: 0.9374 - val_precision: 0.1577 - val_recall: 0.4667 - val_f1_score: 0.1299 - lr: 0.0010
Epoch 3/70
340/340 [==============================] - 2s 7ms/step - loss: 0.5347 - binary_accuracy: 0.9004 - precision: 0.5007 - recall: 0.4662 - f1_score: 0.4255 - val_loss: 0.2338 - val_binary_accuracy: 0.9389 - val_precision: 0.1750 - val_recall: 0.5253 - val_f1_score: 0

In [ ]:
#for channel in np.arange(12):
    
    task_name = f"Channel {channel}"
    
    for times in range(5):
        input_F_X = tf.keras.Input(shape=(500,12,), name='CNN_input_F_X')
        Get_elem = tf.keras.layers.Lambda(lambda x: x[:, :, channel])(input_F_X)
        D_1 = tf.keras.layers.Dense(120, activation = 'linear')(Get_elem)
        D_2 = tf.keras.layers.Dense(90 , activation = 'linear')(D_1)
        D_3 = tf.keras.layers.Dense(60 , activation = 'linear')(D_2)
        D_4 = tf.keras.layers.Dense(30 , activation = 'linear')(D_3)
        outputs = tf.keras.layers.Dense(6, activation='sigmoid')(D_4)
        model = tf.keras.models.Model(inputs=input_F_X
                                      , outputs=outputs)
        #model.summary()

        model.compile(loss=tf.keras.losses.BinaryCrossentropy() #'binary_crossentropy'
                 , optimizer=tf.keras.optimizers.Adam(lr)
                 , metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')
                            , tf.keras.metrics.Precision(name='precision')
                            , tf.keras.metrics.Recall(name='recall')
                            , tf.keras.metrics.F1Score(name='f1_score', average='micro')])

        # CALLBACKS
        callbacks = [ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.1,
                                           patience=6,
                                           min_lr=lr / 100),
                         EarlyStopping(patience=11,  # Patience should be larger than the one in ReduceLROnPlateau
                                       min_delta=0.00001),
                    FitCallback(logs_dir, file_name, val_dataset, task_name)
                    ]

        model.fit( short_train_dataset.batch(8).repeat(70),
                epochs=70,
                initial_epoch=0,
                steps_per_epoch = np.floor(inst_count/8),
                #batch_size = 8,
                callbacks = callbacks,
                #callbacks=[myCallback(init_vars)], 
                validation_data=(val_dataset.batch(8)),
                verbose=1)

In [ ]:
mini_dataset = val_dataset.batch(11).take(1)

print(model.evaluate(val_dataset.batch(11)))
print(np.round(model.predict(mini_dataset ), 2))

for inst in mini_dataset:
    print(inst[1])